#  Коллаборативная фильтрация 

План ноутбука:

1. Вводные по задаче.

2. Библиотеки.

3. Загрузка данных.

4. Задание 1. Создание user-item-матрицы и разбиение данных на тест и контроль результатов.

5. Задание 2. Создание бейзлайнов и расчёт метрик.

* на сырых данных

* на разреженой матрице

* на тестовой матрице

6. Задание 3. Применение метода матричной факторизации и улучшение параметров алгоритма факторизации.

7. Результаты и выводы.

## Вводные по задаче.

Цель практической работы

Научиться:

    применять метод матричного разложения для задачи рекомендательных систем;

    раскладывать матрицу оценок, используя метод матричного разложения;
    
    рассчитывать метрику mAP.

Что входит в работу

    Создать user-item-матрицу и разбить данные на тест и контроль результатов.

    Создать бейзлайны и рассчитать метрики.
    
    Применить метод матричной факторизации и улучшить параметры алгоритма факторизации.

Задание 1. Создание user-item-матрицы и разбиение данных на тест и контроль результатов
Что нужно сделать

Постройте user-item-матрицу на основе данных о прочитанных книгах из таблицы ratings.csv.

    Подключите библиотеку Implicit. 
    
    Для лучшей работы с библиотекой переведите данные рейтинга в бинарные оценки: 1 — книга понравилась, 0 — книга не понравилась. 
    
    Необходимо предсказать, понравится ли конкретная книга заданному пользователю, то есть высчитать implicit-оценки.
    
    Для дальнейшей оценки качества разбейте данные на тест и контроль. 
    
    Так как в данных нет настоящей даты прочтения, воспользуйтесь обходным способом:
    
      Пронумеруйте для каждого пользователя его прочитанные книги согласно расположению книг по порядку.
    
      Переведите номера прочитанных книг в доли по формуле «порядковый номер / общее количество прочитанных пользователем книг».
    
      Определите, какое количество данных вы оставите на обучение, какое — на контроль. Например, 70% книг каждого пользователя — на обучение.
    
    На основе данных для обучения постройте user-item-матрицу.
    
    Посмотрите, сколько в матрице столбцов и строк. Совпадает ли их количество с числом уникальных пользователей и книг?
    
    Выведите список названий книг с высокой оценкой для любого из пользователей.
    
    Запомните индекс пользователя, он пригодится далее.

Задание 2. Создание бейзлайнов и расчёт метрик
Что нужно сделать

Для оценки качества модели создайте несколько простых бейзлайнов.

    Реализуйте функцию, которая подсчитывает метрику AP@K. На вход функция принимает список рекомендаций и список книг, положительно оценённых пользователем (с оценкой 4 или 5).

    Зафиксируйте случайный список из 500 пользовательских индексов. Подсчитайте случайный бейзлайн — для каждого пользователя из списка сгенерируйте случайный ответ из 20 книг. 
    
    Удалите книги, которые пользователь уже читал.

    Рассчитайте метрику AP@10 для каждого из пользователей и усредните её по всем пользователям.
    
    Аналогично постройте бейзлайн из самых популярных книг: посмотрите, какие книги пользователи читали больше всего, и сделайте рекомендации на их основе. Подсчитайте mAP@10.

Задание 3. Применение метода матричной факторизации и улучшение параметров алгоритма факторизации
Что нужно сделать

    Используя результаты, полученные при выполнении заданий 1 и 2, обучите алгоритм ALS из библиотеки Implicit с базовыми параметрами. 
    
    Посмотрите, какие рекомендации вы получите для конкретного пользователя.
    
    Используя полученный ранее случайный список пользователей из задания 2, подсчитайте метрику mAP@10. Превзошла ли она какой-нибудь из бейзлайнов?
    
    Попробуйте использовать более «продвинутые» параметры:

        большее количество факторов — отвечает за размерность векторов;

        больше итераций, чтобы приблизиться к ответу.
        
    Сделайте несколько версий и посмотрите, как будут меняться рекомендации для конкретного пользователя и общая метрика mAP@10 (вы должны превзойти значение 0,02 — это нормально). Сделайте выводы о качестве полученной модели.




## Библиотеки.

In [1]:
import random
import numpy as np
import pandas as pd
import scipy.sparse as sparse # разреж матрица
from pandas.api.types import CategoricalDtype # кодировщик
from collections import Counter
from sklearn.neighbors import NearestNeighbors
import implicit # используется факт взаимодействия с item (1, 0)
from implicit.als import AlternatingLeastSquares

from surprise import Dataset, Reader
from surprise import SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

## Загрузка данных.

In [2]:
books = pd.read_csv('data/books.csv')
ratings = pd.read_csv('data/ratings.csv')

In [3]:
print(ratings.shape)
print(ratings.dtypes)
ratings.isnull().sum()

(5976479, 3)
user_id    int64
book_id    int64
rating     int64
dtype: object


user_id    0
book_id    0
rating     0
dtype: int64

In [4]:
print(books.shape)
print(books.dtypes)
books.isnull().sum()

(10000, 23)
book_id                        int64
goodreads_book_id              int64
best_book_id                   int64
work_id                        int64
books_count                    int64
isbn                          object
isbn13                       float64
authors                       object
original_publication_year    float64
original_title                object
title                         object
language_code                 object
average_rating               float64
ratings_count                  int64
work_ratings_count             int64
work_text_reviews_count        int64
ratings_1                      int64
ratings_2                      int64
ratings_3                      int64
ratings_4                      int64
ratings_5                      int64
image_url                     object
small_image_url               object
dtype: object


book_id                         0
goodreads_book_id               0
best_book_id                    0
work_id                         0
books_count                     0
isbn                          700
isbn13                        585
authors                         0
original_publication_year      21
original_title                585
title                           0
language_code                1084
average_rating                  0
ratings_count                   0
work_ratings_count              0
work_text_reviews_count         0
ratings_1                       0
ratings_2                       0
ratings_3                       0
ratings_4                       0
ratings_5                       0
image_url                       0
small_image_url                 0
dtype: int64

## Задание 1. Создание user-item-матрицы и разбиение данных на тест и контроль результатов.

In [5]:
ratings = pd.read_csv('data/ratings.csv')

In [6]:
# смотрим значения
ratings['rating'].value_counts()

rating
4    2139018
5    1983093
3    1370916
2     359257
1     124195
Name: count, dtype: int64

### Подготовка и разделение данных.

In [7]:
# Преобразование в бинарные оценки
#ratings['binary_rating'] = (ratings['rating'] > 0).astype(int)
# смотрим значения
#ratings['binary_rating'].value_counts()

In [8]:
# делаем признак бинарным
threshold = 3 # устанавливаем пороговое значение
ratings['binary_rating'] = (ratings['rating'] > threshold).astype(int)
ratings['binary_rating'].value_counts()

binary_rating
1    4122111
0    1854368
Name: count, dtype: int64

In [9]:
# Нумерация книг для каждого пользователя
ratings['book_rank'] = ratings.groupby('user_id')['binary_rating'].rank(method='first')
ratings['book_rank']

0           57.0
1           12.0
2           13.0
3           14.0
4            1.0
           ...  
5976474    131.0
5976475    132.0
5976476    133.0
5976477    134.0
5976478    135.0
Name: book_rank, Length: 5976479, dtype: float64

In [10]:
# Перевод в доли
ratings['book_fraction'] = ratings['book_rank'] / ratings.groupby('user_id')['book_rank'].transform('max')
ratings['book_fraction'].value_counts()

book_fraction
1.000000    53424
0.500000    26817
0.666667    17693
0.333333    17693
0.750000    13323
            ...  
0.563452        2
0.568528        2
0.573604        2
0.578680        2
0.482234        2
Name: count, Length: 12172, dtype: int64

In [11]:
train_data = ratings[ratings['book_fraction'] <= 0.7]
test_data = ratings[ratings['book_fraction'] > 0.7]

In [12]:
# берем уникальные значения
user_index = train_data.user_id.unique()
books_index = train_data.book_id.unique()

# делаем матрицу
rows = train_data['user_id'].astype(CategoricalDtype(categories=user_index)).cat.codes # строки юзеры
cols = train_data['book_id'].astype(CategoricalDtype(categories=books_index)).cat.codes # колонки итемы

matrix_csr = sparse.csr_matrix((train_data['binary_rating'], (rows, cols)), shape=(len(user_index), len(books_index))) # разреженая матрица
#matrix_csr = matrix.toarray()

In [13]:
# тестовая матрица
user_index_test = test_data.user_id.unique()
books_index_test = test_data.book_id.unique()

# делаем матрицу
rows_test = test_data['user_id'].astype(CategoricalDtype(categories=user_index)).cat.codes # строки юзеры
cols_test = test_data['book_id'].astype(CategoricalDtype(categories=books_index)).cat.codes # колонки итемы

matrix_test = sparse.csr_matrix((test_data['binary_rating'], (rows_test, cols_test)), shape=(len(user_index_test), len(books_index_test))) # разреженая матрица
#matrix_test = matrix_t.toarray()

In [14]:
# Проверьте количество строк и столбцов в матрице:
num_users, num_books = matrix_csr.shape
print(f"Количество пользователей: {num_users}, Количество книг: {num_books}")

Количество пользователей: 53424, Количество книг: 10000


In [15]:
ratings['user_id'].nunique()

53424

In [16]:
def recomend_symply(user, matrix_csr):
    matrix_csr = matrix_csr.toarray()
    for ind, score in enumerate(matrix_csr[user]):
        if score>=1:
            book_id = books_index[ind]
            print(score, books[books['book_id']==book_id].original_title.values[0])
            

In [17]:
user = 50
recomend_symply(user, matrix_csr)

1 The Sun Also Rises
1 The Catcher in the Rye
1 Slaughterhouse-Five, or The Children's Crusade: A Duty-Dance with Death 
1 Life of Pi
1 Animal Farm: A Fairy Story
1 Nineteen Eighty-Four
1 The Sea
1 Of Mice and Men 
1 Das Parfum. Die Geschichte eines Mörders
1 The Secret History
1 Motherless Brooklyn
1 In Cold Blood
1 The Poisonwood Bible
1 Frankenstein; or, The Modern Prometheus
1 Howl and Other Poems 
1 The Little Friend
1 Fahrenheit 451
1 On the Road
1 Nesnesitelná lehkost bytí
1 The Bean Trees
1 L'Amant
1 Candide
1 Angela's Ashes: A Memoir
1 Still Life with Woodpecker
1 Le Petit Prince
1 Siddhartha
1 The Crucible
1 The Kitchen God's Wife
1 The Grapes of Wrath
1 Demian: Die Geschichte einer Jugend
1 Voyage au bout de la nuit
1 The English Patient
1 East of Eden  
1 The Winter of our Discontent
1 Invisible Man
1 What We Talk About When We Talk About Love: Stories
1 Will You Please Be Quiet, Please?
1 Native Son 
1 The Jungle
1 The Garden of Eden
1  Carter Beats the Devil
1 Skinny Legs

## Задание 2. Создание бейзлайнов и расчёт метрик.

In [18]:
def apk(actual, predicted, k=10):
    """
    Вычисляет AP@K для одного пользователя.

    :param actual: Список книг, которые пользователь оценил на 4 или 5.
    :param predicted: Список рекомендованных книг.
    :param k: Количество рекомендаций для рассмотрения.
    :return: Значение AP@K.
    """
    if len(predicted) > k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i + 1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

In [19]:
# Зафиксируем случайный список из 500 пользователей
random_users = random.sample(range(num_users), 500)

### на сырых данных.

In [20]:
# Генерация случайного бейзлайна без учета рейтинга книг
random_recommendations = {}
for user in random_users:
    # Получаем уже прочитанные книги пользователем
    read_books = set(ratings[ratings['user_id'] == user]['book_id'])
    
    # Генерируем случайные рекомендации из всех книг, исключая прочитанные
    all_books = set(books['book_id']) - read_books
    recommendations = random.sample(list(all_books), min(10, len(all_books)))  # Преобразуем множество в список
    
    random_recommendations[user] = recommendations



# Подсчет AP@10 для случайного бейзлайна
ap_scores_random = []
for user in random_users:
    # Получаем релевантные книги (с оценкой 4 или 5)
    relevant_books = ratings[(ratings['user_id'] == user) & (ratings['rating'] >= 4)]['book_id'].tolist()
    
    # Получаем рекомендации
    recommendations = random_recommendations[user]
    
    # Рассчитываем AP@10
    ap_score = apk(relevant_books, recommendations, k=10)
    ap_scores_random.append(ap_score)

# Усреднение AP@10 по всем пользователям
mean_ap_random = sum(ap_scores_random) / len(ap_scores_random)
print(f"Средний AP@10 для случайного бейзлайна: {mean_ap_random:.4f}")



Средний AP@10 для случайного бейзлайна: 0.0000


In [21]:
# проверка получения 0

# Проверка наличия релевантных книг
for user in random_users:
    relevant_books = ratings[(ratings['user_id'] == user) & (ratings['rating'] >= 4)]['book_id'].tolist()
    if not relevant_books:
        print(f"У пользователя {user} нет релевантных книг (оценка 4 или 5).")


for user in random_users:
    recommendations = random_recommendations[user]
    if not recommendations:
        print(f"Для пользователя {user} не удалось сгенерировать рекомендации.")

### На разреженой матрице.

In [22]:
# сделаем то-же самое на разреженой матрице


# Генерация случайного бейзлайна без учета рейтинга книг
random_recommendations = {}
for user in random_users:
    # Получаем индекс пользователя в разреженной матрице
    user_index = user  
    
    # Получаем уже прочитанные книги пользователем
    read_books_indices = matrix_csr[user_index].nonzero()[0]  # Индексы прочитанных книг
    read_books = set(books['book_id'].iloc[read_books_indices])  # Преобразуем индексы в book_id
    
    # Генерируем случайные рекомендации из всех книг, исключая прочитанные
    all_books = set(books['book_id']) - read_books
    
    if len(all_books) > 0:
        recommendations = random.sample(list(all_books), min(10, len(all_books)))  # Преобразуем множество в список
    else:
        recommendations = []  # Если нет доступных книг для рекомендации
    
    random_recommendations[user] = recommendations

# Подсчет AP@10 для случайного бейзлайна
ap_scores_random = []
for user in random_users:
    # Получаем релевантные книги (с оценкой 4 или 5)
    relevant_books = ratings[(ratings['user_id'] == user) & (ratings['rating'] >= 4)]['book_id'].tolist()
    
    # Получаем рекомендации
    recommendations = random_recommendations[user]
    
    # Рассчитываем AP@10
    ap_score = apk(relevant_books, recommendations, k=10)
    ap_scores_random.append(ap_score)

# Усреднение AP@10 по всем пользователям
mean_ap_random_matrix = sum(ap_scores_random) / len(ap_scores_random) if ap_scores_random else 0
print(f"Средний AP@10 для случайного бейзлайна: {mean_ap_random:.4f}")
print(f"Средний AP@10 для случайного бейзлайна matrix: {mean_ap_random_matrix:.4f}")


Средний AP@10 для случайного бейзлайна: 0.0000
Средний AP@10 для случайного бейзлайна matrix: 0.0023


### На тестовой матрице.

In [23]:
# На разреженой тестовой матрице


# Генерация случайного бейзлайна без учета рейтинга книг
random_recommendations = {}
for user in random_users:
    # Получаем индекс пользователя в разреженной матрице
    user_index = user  
    
    # Проверяем, существует ли пользователь в разреженной матрице
    if user_index >= matrix_test.shape[0]:
        continue  # Если индекс выходит за пределы, пропускаем пользователя
    
    # Получаем уже прочитанные книги пользователем
    read_books_indices = matrix_test[user_index].nonzero()[0]  # Индексы прочитанных книг
    
    # Проверка на наличие прочитанных книг
    if read_books_indices.size == 0:
        read_books = set()  # Если нет прочитанных книг, создаем пустое множество
    else:
        read_books = set(books['book_id'].iloc[read_books_indices])  # Преобразуем индексы в book_id
    
    # Генерируем случайные рекомендации из всех книг, исключая прочитанные
    all_books = set(books['book_id']) - read_books
    
    if len(all_books) > 0:
        recommendations = random.sample(list(all_books), min(10, len(all_books)))  # Преобразуем множество в список
    else:
        recommendations = []  # Если нет доступных книг для рекомендации
    
    random_recommendations[user] = recommendations

# Подсчет AP@10 для случайного бейзлайна
ap_scores_random = []
for user in random_users:
    # Получаем релевантные книги (с оценкой 4 или 5)
    relevant_books = ratings[(ratings['user_id'] == user) & (ratings['rating'] >= 4)]['book_id'].tolist()
    
    # Получаем рекомендации
    recommendations = random_recommendations[user]
    
    # Рассчитываем AP@10
    ap_score = apk(relevant_books, recommendations, k=10)
    ap_scores_random.append(ap_score)

# Усреднение AP@10 по всем пользователям
mean_ap_random_matrix_test = sum(ap_scores_random) / len(ap_scores_random) if ap_scores_random else 0
print(f"Средний AP@10 для случайного бейзлайна: {mean_ap_random:.4f}")
print(f"Средний AP@10 для случайного бейзлайна matrix: {mean_ap_random_matrix:.4f}")
print(f"Средний AP@10 для случайного бейзлайна на matrix_test: {mean_ap_random_matrix_test:.4f}")


Средний AP@10 для случайного бейзлайна: 0.0000
Средний AP@10 для случайного бейзлайна matrix: 0.0023
Средний AP@10 для случайного бейзлайна на matrix_test: 0.0025


### на сырых данных.

In [24]:
# Подсчет популярности книг
popular_books = ratings.groupby('book_id').size().nlargest(20).index.tolist()
# Генерация рекомендаций на основе популярных книг для каждого пользователя
popular_recommendations = {user: popular_books for user in random_users}
popular_recommendations

# Генерация рекомендаций на основе популярных книг для каждого пользователя
popular_recommendations = {user: popular_books for user in random_users}

# Подсчет AP@10 для бейзлайна на основе популярных книг
ap_scores_popular = []
for user in random_users:
    # Получаем релевантные книги (с оценкой 4 или 5)
    relevant_books = ratings[(ratings['user_id'] == user) & (ratings['rating'] >= 4)]['book_id'].tolist()
    
    # Получаем рекомендации
    recommendations = popular_recommendations[user]
    
    # Рассчитываем AP@10
    ap_score = apk(relevant_books, recommendations, k=10)
    ap_scores_popular.append(ap_score)

# Усреднение AP@10 по всем пользователям
mean_ap_popular = sum(ap_scores_popular) / len(ap_scores_popular)
print(f"Средний AP@10 для бейзлайна на основе популярных книг: {mean_ap_popular:.4f}")

Средний AP@10 для бейзлайна на основе популярных книг: 0.1595


### На разреженой матрице.

In [25]:
# то-же на разреженой матрице

# Подсчет популярности книг
popular_books = ratings.groupby('book_id').size().nlargest(20).index.tolist()

# Генерация рекомендаций на основе популярных книг для каждого пользователя
popular_recommendations = {}
for user in random_users:
    # Получаем индекс пользователя в разреженной матрице
    user_index = user
    
    # Проверяем, существует ли пользователь в разреженной матрице
    if user_index >= matrix_csr.shape[0]:
        continue  # Если индекс выходит за пределы, пропускаем пользователя
    
    # Получаем уже прочитанные книги пользователем
    read_books_indices = matrix_csr[user_index].nonzero()[0]  # Индексы прочитанных книг
    
    # Преобразуем индексы в book_id
    read_books = set(books['book_id'].iloc[read_books_indices]) if read_books_indices.size > 0 else set()
    
    # Генерируем рекомендации на основе популярных книг, исключая прочитанные
    recommendations = [book for book in popular_books if book not in read_books]
    
    # Ограничиваем количество рекомендаций до 10
    popular_recommendations[user] = recommendations[:10]

# Подсчет AP@10 для бейзлайна на основе популярных книг
ap_scores_popular = []
for user in random_users:
    # Получаем релевантные книги (с оценкой 4 или 5)
    relevant_books = ratings[(ratings['user_id'] == user) & (ratings['rating'] >= 4)]['book_id'].tolist()
    
    # Получаем рекомендации
    recommendations = popular_recommendations.get(user, [])
    
    # Рассчитываем AP@10
    ap_score = apk(relevant_books, recommendations, k=10)
    ap_scores_popular.append(ap_score)

# Усреднение AP@10 по всем пользователям
mean_ap_popular_matrix = sum(ap_scores_popular) / len(ap_scores_popular) if ap_scores_popular else 0
print(f"Средний AP@10 для бейзлайна на основе популярных книг: {mean_ap_popular:.4f}")
print(f"Средний AP@10 для бейзлайна на основе популярных книг matrix: {mean_ap_popular_matrix:.4f}")


Средний AP@10 для бейзлайна на основе популярных книг: 0.1595
Средний AP@10 для бейзлайна на основе популярных книг matrix: 0.1511


### На тестовой матрице.

In [26]:
# на тестовой выборке

# Подсчет популярности книг
popular_books = ratings.groupby('book_id').size().nlargest(20).index.tolist()

# Генерация рекомендаций на основе популярных книг для каждого пользователя
popular_recommendations = {}
for user in random_users:
    # Получаем индекс пользователя в разреженной матрице
    user_index = user
    
    # Проверяем, существует ли пользователь в разреженной матрице
    if user_index >= matrix_test.shape[0]:
        continue  # Если индекс выходит за пределы, пропускаем пользователя
    
    # Получаем уже прочитанные книги пользователем
    read_books_indices = matrix_test[user_index].nonzero()[0]  # Индексы прочитанных книг
    
    # Преобразуем индексы в book_id
    read_books = set(books['book_id'].iloc[read_books_indices]) if read_books_indices.size > 0 else set()
    
    # Генерируем рекомендации на основе популярных книг, исключая прочитанные
    recommendations = [book for book in popular_books if book not in read_books]
    
    # Ограничиваем количество рекомендаций до 10
    popular_recommendations[user] = recommendations[:10]

# Подсчет AP@10 для бейзлайна на основе популярных книг
ap_scores_popular = []
for user in random_users:
    # Получаем релевантные книги (с оценкой 4 или 5)
    relevant_books = ratings[(ratings['user_id'] == user) & (ratings['rating'] >= 4)]['book_id'].tolist()
    
    # Получаем рекомендации
    recommendations = popular_recommendations.get(user, [])
    
    # Рассчитываем AP@10
    ap_score = apk(relevant_books, recommendations, k=10)
    ap_scores_popular.append(ap_score)

# Усреднение AP@10 по всем пользователям
mean_ap_popular_test = sum(ap_scores_popular) / len(ap_scores_popular) if ap_scores_popular else 0
print(f"Средний AP@10 для бейзлайна на основе популярных книг: {mean_ap_popular:.4f}")
print(f"Средний AP@10 для бейзлайна на основе популярных книг matrix: {mean_ap_popular_matrix:.4f}")
print(f"Средний AP@10 для бейзлайна на основе популярных книг test: {mean_ap_popular_test:.4f}")

Средний AP@10 для бейзлайна на основе популярных книг: 0.1595
Средний AP@10 для бейзлайна на основе популярных книг matrix: 0.1511
Средний AP@10 для бейзлайна на основе популярных книг test: 0.1516


## Задание 3. Применение метода матричной факторизации и улучшение параметров алгоритма факторизации.

In [27]:
model_base = implicit.als.AlternatingLeastSquares()

model_base.fit(sparse.csr_matrix(matrix_csr))

/home/leostuchchi/anaconda3/lib/python3.11/site-packages/implicit/cpu/als.py:95: RuntimeWarning: Intel MKL BLAS is configured to use 2 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'MKL_NUM_THREADS=1' or by callng 'threadpoolctl.threadpool_limits(1, "blas")'. Having MKL use a threadpool can lead to severe performance issues
  check_blas_config()


  0%|          | 0/15 [00:00<?, ?it/s]

In [28]:
# у модели есть метод recommend для юзер 50
recommendations = model_base.recommend(50, sparse.csr_matrix(matrix_csr)[50])
# выводим рекомендации для юзер 50
recommendations # items индексы и оценки

for ind in recommendations[0]:
    book_id = books_index[ind]
    print(books[books['book_id']==book_id].original_title.values[0])

Cien años de soledad
The Handmaid's Tale
Lolita
The Road
Middlesex
Brave New World
O Alquimista
A Wrinkle in Time
Cat's Cradle
Catch-22


In [29]:
model_base.recommend(user, sparse.csr_matrix(matrix_csr)[user])[0]

array([ 143,   17,   19,   16,   89, 1130, 2206,   15,  110,   91],
      dtype=int32)

In [30]:
# Генерация случайного бейзлайна без учета рейтинга книг
random_recommendations = {}
for user in random_users:
    # Получаем уже прочитанные книги пользователем
    read_books = set(ratings[ratings['user_id'] == user]['book_id'])
    
    # Генерируем случайные рекомендации из всех книг, исключая прочитанные
    all_books = set(books['book_id']) - read_books
    #recommendations = random.sample(list(all_books), min(10, len(all_books)))  # Преобразуем множество в список
    recommendations = model_base.recommend(user, sparse.csr_matrix(matrix_csr)[user])[0]
    
    random_recommendations[user] = recommendations



# Подсчет AP@10 для случайного бейзлайна
ap_scores_random = []
for user in random_users:
    # Получаем релевантные книги (с оценкой 4 или 5)
    relevant_books = ratings[(ratings['user_id'] == user) & (ratings['rating'] >= 4)]['book_id'].tolist()
    
    # Получаем рекомендации
    recommendations = random_recommendations[user]
    
    # Рассчитываем AP@10
    ap_score = apk(relevant_books, recommendations, k=10)
    ap_scores_random.append(ap_score)

# Усреднение AP@10 по всем пользователям
mean_ap_random_model_base = sum(ap_scores_random) / len(ap_scores_random)

print(f"Средний AP@10 для случайного бейзлайна: {mean_ap_random:.4f}")
print(f"Средний AP@10 для случайного бейзлайна matrix: {mean_ap_random_matrix:.4f}")
print(f"Средний AP@10 для случайного бейзлайна на matrix_test: {mean_ap_random_matrix_test:.4f}")
print(f"Средний AP@10 для случайного бейзлайна model_base_train: {mean_ap_random_model_base:.4f}")

Средний AP@10 для случайного бейзлайна: 0.0000
Средний AP@10 для случайного бейзлайна matrix: 0.0023
Средний AP@10 для случайного бейзлайна на matrix_test: 0.0025
Средний AP@10 для случайного бейзлайна model_base_train: 0.0135


### На тестовых данных

In [31]:


# Генерация случайного бейзлайна без учета рейтинга книг
random_recommendations = {}
for user in random_users:
    # Получаем индекс пользователя в разреженной матрице
    user_index = user  # Предполагается, что user является индексом
    
    # Проверяем, существует ли пользователь в разреженной матрице
    if user_index >= matrix_test.shape[0]:
        continue  # Если индекс выходит за пределы, пропускаем пользователя
    
    # Получаем уже прочитанные книги пользователем
    read_books_indices = matrix_test[user_index].nonzero()[0]  # Индексы прочитанных книг
    
    # Преобразуем индексы в book_id
    read_books = set(books['book_id'].iloc[read_books_indices]) if read_books_indices.size > 0 else set()
    
    # Генерируем случайные рекомендации из всех книг, исключая прочитанные
    all_books = set(books['book_id']) - read_books
    
    if len(all_books) > 0:
        recommendations = random.sample(list(all_books), min(10, len(all_books)))  # Преобразуем множество в список
    else:
        recommendations = []  # Если нет доступных книг для рекомендации
    
    random_recommendations[user] = recommendations

# Подсчет AP@10 для случайного бейзлайна
ap_scores_random = []
for user in random_users:
    # Получаем релевантные книги (с оценкой 4 или 5)
    relevant_books = ratings[(ratings['user_id'] == user) & (ratings['rating'] >= 4)]['book_id'].tolist()
    
    # Получаем рекомендации
    recommendations = random_recommendations.get(user, [])
    
    # Рассчитываем AP@10
    ap_score = apk(relevant_books, recommendations, k=10)
    ap_scores_random.append(ap_score)

# Усреднение AP@10 по всем пользователям
mean_ap_random_model_base_test = sum(ap_scores_random) / len(ap_scores_random) if ap_scores_random else 0

# Вывод результатов
print(f"Средний AP@10 для случайного бейзлайна: {mean_ap_random:.4f}")
print(f"Средний AP@10 для случайного бейзлайна matrix: {mean_ap_random_matrix:.4f}")
print(f"Средний AP@10 для случайного бейзлайна на matrix_test: {mean_ap_random_matrix_test:.4f}")
print(f"Средний AP@10 для случайного бейзлайна model_base_train: {mean_ap_random_model_base:.4f}")
print(f"Средний AP@10 для случайного бейзлайна model_base_test: {mean_ap_random_model_base_test:.4f}")


Средний AP@10 для случайного бейзлайна: 0.0000
Средний AP@10 для случайного бейзлайна matrix: 0.0023
Средний AP@10 для случайного бейзлайна на matrix_test: 0.0025
Средний AP@10 для случайного бейзлайна model_base_train: 0.0135
Средний AP@10 для случайного бейзлайна model_base_test: 0.0023


### Популярность книг.

### На трейне

In [32]:

ap_scores_popular = []
for user in random_users:
    # Получаем релевантные книги (с оценкой 4 или 5)
    relevant_books = ratings[(ratings['user_id'] == user) & (ratings['rating'] >= 4)]['book_id'].tolist()
    
    # Получаем рекомендации
    recommendations = model_base.recommend(user, sparse.csr_matrix(matrix_csr)[user])[0]#popular_recommendations[user]
    
    # Рассчитываем AP@10
    ap_score = apk(relevant_books, recommendations, k=10)
    ap_scores_popular.append(ap_score)

# Усреднение AP@10 по всем пользователям
mean_ap_popular_model_base = sum(ap_scores_popular) / len(ap_scores_popular)

print(f"Средний AP@10 для бейзлайна на основе популярных книг: {mean_ap_popular:.4f}")
print(f"Средний AP@10 для бейзлайна на основе популярных книг matrix: {mean_ap_popular_matrix:.4f}")
print(f"Средний AP@10 для бейзлайна на основе популярных книг test: {mean_ap_popular_test:.4f}")
print(f"Средний AP@10 для бейзлайна на основе популярных книг model_base_train: {mean_ap_popular_model_base:.4f}")

Средний AP@10 для бейзлайна на основе популярных книг: 0.1595
Средний AP@10 для бейзлайна на основе популярных книг matrix: 0.1511
Средний AP@10 для бейзлайна на основе популярных книг test: 0.1516
Средний AP@10 для бейзлайна на основе популярных книг model_base_train: 0.0135


### На тесте.

In [33]:

ap_scores_popular = []
for user in random_users:
    # Получаем релевантные книги (с оценкой 4 или 5)
    relevant_books = ratings[(ratings['user_id'] == user) & (ratings['rating'] >= 4)]['book_id'].tolist()
    
    # Получаем рекомендации
    recommendations = model_base.recommend(user, sparse.csr_matrix(matrix_test)[user])[0]#popular_recommendations[user]
    
    # Рассчитываем AP@10
    ap_score = apk(relevant_books, recommendations, k=10)
    ap_scores_popular.append(ap_score)

# Усреднение AP@10 по всем пользователям
mean_ap_popular_model_base_test = sum(ap_scores_popular) / len(ap_scores_popular)

print(f"Средний AP@10 для бейзлайна на основе популярных книг: {mean_ap_popular:.4f}")
print(f"Средний AP@10 для бейзлайна на основе популярных книг matrix: {mean_ap_popular_matrix:.4f}")
print(f"Средний AP@10 для бейзлайна на основе популярных книг test: {mean_ap_popular_test:.4f}")
print(f"Средний AP@10 для бейзлайна на основе популярных книг model_base_train: {mean_ap_popular_model_base:.4f}")
print(f"Средний AP@10 для бейзлайна на основе популярных книг model_base_test: {mean_ap_popular_model_base_test:.4f}")

Средний AP@10 для бейзлайна на основе популярных книг: 0.1595
Средний AP@10 для бейзлайна на основе популярных книг matrix: 0.1511
Средний AP@10 для бейзлайна на основе популярных книг test: 0.1516
Средний AP@10 для бейзлайна на основе популярных книг model_base_train: 0.0135
Средний AP@10 для бейзлайна на основе популярных книг model_base_test: 0.0207


### Улучшеная модель

In [34]:
# Создание модели ALS с продвинутыми параметрами
model_advanced = implicit.als.AlternatingLeastSquares(
    factors=10,          # Увеличенное количество факторов
    regularization=0.1,   # Регуляризация
    iterations=30,        # Увеличенное количество итераций
    alpha=5.0            # Увеличение веса для бинарных оценок
)

In [35]:
# Обучение модели
model_advanced.fit(sparse.csr_matrix(matrix_csr))

  0%|          | 0/30 [00:00<?, ?it/s]

In [36]:
# Генерация случайного бейзлайна без учета рейтинга книг
random_recommendations = {}
for user in random_users:
    # Получаем уже прочитанные книги пользователем
    read_books = set(ratings[ratings['user_id'] == user]['book_id'])
    
    # Генерируем случайные рекомендации из всех книг, исключая прочитанные
    all_books = set(books['book_id']) - read_books
    #recommendations = random.sample(list(all_books), min(10, len(all_books)))  # Преобразуем множество в список
    recommendations = model_advanced.recommend(user, sparse.csr_matrix(matrix_csr)[user])[0]
    
    random_recommendations[user] = recommendations



# Подсчет AP@10 для случайного бейзлайна
ap_scores_random = []
for user in random_users:
    # Получаем релевантные книги (с оценкой 4 или 5)
    relevant_books = ratings[(ratings['user_id'] == user) & (ratings['rating'] >= 4)]['book_id'].tolist()
    
    # Получаем рекомендации
    recommendations = random_recommendations[user]
    
    # Рассчитываем AP@10
    ap_score = apk(relevant_books, recommendations, k=10)
    ap_scores_random.append(ap_score)

# Усреднение AP@10 по всем пользователям
mean_ap_random_model_advanced = sum(ap_scores_random) / len(ap_scores_random)

print(f"Средний AP@10 для случайного бейзлайна: {mean_ap_random:.4f}")
print(f"Средний AP@10 для случайного бейзлайна matrix: {mean_ap_random_matrix:.4f}")
print(f"Средний AP@10 для случайного бейзлайна на matrix_test: {mean_ap_random_matrix_test:.4f}")
print(f"Средний AP@10 для случайного бейзлайна model_base_train: {mean_ap_random_model_base:.4f}")
print(f"Средний AP@10 для случайного бейзлайна model_base_test: {mean_ap_random_model_base_test:.4f}")
print(f"Средний AP@10 для случайного бейзлайна model_advanced_train: {mean_ap_random_model_advanced:.4f}")

Средний AP@10 для случайного бейзлайна: 0.0000
Средний AP@10 для случайного бейзлайна matrix: 0.0023
Средний AP@10 для случайного бейзлайна на matrix_test: 0.0025
Средний AP@10 для случайного бейзлайна model_base_train: 0.0135
Средний AP@10 для случайного бейзлайна model_base_test: 0.0023
Средний AP@10 для случайного бейзлайна model_advanced_train: 0.0216


In [37]:
# На тесте
# Генерация случайного бейзлайна без учета рейтинга книг
random_recommendations = {}
for user in random_users:
    # Получаем уже прочитанные книги пользователем
    read_books = set(ratings[ratings['user_id'] == user]['book_id'])
    
    # Генерируем случайные рекомендации из всех книг, исключая прочитанные
    all_books = set(books['book_id']) - read_books
    #recommendations = random.sample(list(all_books), min(10, len(all_books)))  # Преобразуем множество в список
    recommendations = model_advanced.recommend(user, sparse.csr_matrix(matrix_test)[user])[0]
    
    random_recommendations[user] = recommendations



# Подсчет AP@10 для случайного бейзлайна
ap_scores_random = []
for user in random_users:
    # Получаем релевантные книги (с оценкой 4 или 5)
    relevant_books = ratings[(ratings['user_id'] == user) & (ratings['rating'] >= 4)]['book_id'].tolist()
    
    # Получаем рекомендации
    recommendations = random_recommendations[user]
    
    # Рассчитываем AP@10
    ap_score = apk(relevant_books, recommendations, k=10)
    ap_scores_random.append(ap_score)

# Усреднение AP@10 по всем пользователям
mean_ap_random_model_advanced_test = sum(ap_scores_random) / len(ap_scores_random)

print(f"Средний AP@10 для случайного бейзлайна: {mean_ap_random:.4f}")
print(f"Средний AP@10 для случайного бейзлайна matrix: {mean_ap_random_matrix:.4f}")
print(f"Средний AP@10 для случайного бейзлайна на matrix_test: {mean_ap_random_matrix_test:.4f}")
print(f"Средний AP@10 для случайного бейзлайна model_base_train: {mean_ap_random_model_base:.4f}")
print(f"Средний AP@10 для случайного бейзлайна model_base_test: {mean_ap_random_model_base_test:.4f}")
print(f"Средний AP@10 для случайного бейзлайна model_advanced_train: {mean_ap_random_model_advanced:.4f}")
print(f"Средний AP@10 для случайного бейзлайна model_advanced_test: {mean_ap_random_model_advanced_test:.4f}")

Средний AP@10 для случайного бейзлайна: 0.0000
Средний AP@10 для случайного бейзлайна matrix: 0.0023
Средний AP@10 для случайного бейзлайна на matrix_test: 0.0025
Средний AP@10 для случайного бейзлайна model_base_train: 0.0135
Средний AP@10 для случайного бейзлайна model_base_test: 0.0023
Средний AP@10 для случайного бейзлайна model_advanced_train: 0.0216
Средний AP@10 для случайного бейзлайна model_advanced_test: 0.0276


In [38]:
# Подсчет AP@10 для бейзлайна на основе популярных книг
ap_scores_popular = []
for user in random_users:
    # Получаем релевантные книги (с оценкой 4 или 5)
    relevant_books = ratings[(ratings['user_id'] == user) & (ratings['rating'] >= 4)]['book_id'].tolist()
    
    # Получаем рекомендации
    recommendations = model_advanced.recommend(user, sparse.csr_matrix(matrix_csr)[user])[0]#popular_recommendations[user]
    
    # Рассчитываем AP@10
    ap_score = apk(relevant_books, recommendations, k=10)
    ap_scores_popular.append(ap_score)

# Усреднение AP@10 по всем пользователям
mean_ap_popular_model_advabced = sum(ap_scores_popular) / len(ap_scores_popular)
print(f"Средний AP@10 для бейзлайна на основе популярных книг: {mean_ap_popular:.4f}")
print(f"Средний AP@10 для бейзлайна на основе популярных книг matrix: {mean_ap_popular_matrix:.4f}")
print(f"Средний AP@10 для бейзлайна на основе популярных книг test: {mean_ap_popular_test:.4f}")
print(f"Средний AP@10 для бейзлайна на основе популярных книг model_base_train: {mean_ap_popular_model_base:.4f}")
print(f"Средний AP@10 для бейзлайна на основе популярных книг model_base_test: {mean_ap_popular_model_base_test:.4f}")
print(f"Средний AP@10 для бейзлайна на основе популярных книг model_advanced_train: {mean_ap_popular_model_advabced:.4f}")

Средний AP@10 для бейзлайна на основе популярных книг: 0.1595
Средний AP@10 для бейзлайна на основе популярных книг matrix: 0.1511
Средний AP@10 для бейзлайна на основе популярных книг test: 0.1516
Средний AP@10 для бейзлайна на основе популярных книг model_base_train: 0.0135
Средний AP@10 для бейзлайна на основе популярных книг model_base_test: 0.0207
Средний AP@10 для бейзлайна на основе популярных книг model_advanced_train: 0.0216


In [39]:
# Подсчет AP@10 для бейзлайна на основе популярных книг
ap_scores_popular = []
for user in random_users:
    # Получаем релевантные книги (с оценкой 4 или 5)
    relevant_books = ratings[(ratings['user_id'] == user) & (ratings['rating'] >= 4)]['book_id'].tolist()
    
    # Получаем рекомендации
    recommendations = model_advanced.recommend(user, sparse.csr_matrix(matrix_test)[user])[0]#popular_recommendations[user]
    
    # Рассчитываем AP@10
    ap_score = apk(relevant_books, recommendations, k=10)
    ap_scores_popular.append(ap_score)

# Усреднение AP@10 по всем пользователям
mean_ap_popular_model_advabced_test2 = sum(ap_scores_popular) / len(ap_scores_popular)
print(f"Средний AP@10 для бейзлайна на основе популярных книг: {mean_ap_popular:.4f}")
print(f"Средний AP@10 для бейзлайна на основе популярных книг matrix: {mean_ap_popular_matrix:.4f}")
print(f"Средний AP@10 для бейзлайна на основе популярных книг test: {mean_ap_popular_test:.4f}")
print(f"Средний AP@10 для бейзлайна на основе популярных книг model_base_train: {mean_ap_popular_model_base:.4f}")
print(f"Средний AP@10 для бейзлайна на основе популярных книг model_base_test: {mean_ap_popular_model_base_test:.4f}")
print(f"Средний AP@10 для бейзлайна на основе популярных книг model_advanced_train: {mean_ap_popular_model_advabced:.4f}")
print(f"Средний AP@10 для бейзлайна на основе популярных книг model_advanced_test: {mean_ap_popular_model_advabced_test2:.4f}")

Средний AP@10 для бейзлайна на основе популярных книг: 0.1595
Средний AP@10 для бейзлайна на основе популярных книг matrix: 0.1511
Средний AP@10 для бейзлайна на основе популярных книг test: 0.1516
Средний AP@10 для бейзлайна на основе популярных книг model_base_train: 0.0135
Средний AP@10 для бейзлайна на основе популярных книг model_base_test: 0.0207
Средний AP@10 для бейзлайна на основе популярных книг model_advanced_train: 0.0216
Средний AP@10 для бейзлайна на основе популярных книг model_advanced_test: 0.0276


## Результаты и выводы.

In [40]:
print('Результаты эксперементов для случайного бейзлайна без учета рейтингов книг')
print('*'*10)
print(f"Средний AP@10 для случайного бейзлайна: {mean_ap_random:.4f}")
print(f"Средний AP@10 для случайного бейзлайна matrix: {mean_ap_random_matrix:.4f}")
print(f"Средний AP@10 для случайного бейзлайна на matrix_test: {mean_ap_random_matrix_test:.4f}")
print(f"Средний AP@10 для случайного бейзлайна model_base_train: {mean_ap_random_model_base:.4f}")
print(f"Средний AP@10 для случайного бейзлайна model_base_test: {mean_ap_random_model_base_test:.4f}")
print(f"Средний AP@10 для случайного бейзлайна model_advanced_train: {mean_ap_random_model_advanced:.4f}")
print(f"Средний AP@10 для случайного бейзлайна model_advanced_test: {mean_ap_random_model_advanced_test:.4f}")
print('*'*10)
print('Результаты эксперементов для бейзлайна на основе популярных книг')
print('*'*10)
print(f"Средний AP@10 для бейзлайна на основе популярных книг: {mean_ap_popular:.4f}")
print(f"Средний AP@10 для бейзлайна на основе популярных книг matrix: {mean_ap_popular_matrix:.4f}")
print(f"Средний AP@10 для бейзлайна на основе популярных книг test: {mean_ap_popular_test:.4f}")
print(f"Средний AP@10 для бейзлайна на основе популярных книг model_base_train: {mean_ap_popular_model_base:.4f}")
print(f"Средний AP@10 для бейзлайна на основе популярных книг model_base_test: {mean_ap_popular_model_base_test:.4f}")
print(f"Средний AP@10 для бейзлайна на основе популярных книг model_advanced_train: {mean_ap_popular_model_advabced:.4f}")
print(f"Средний AP@10 для бейзлайна на основе популярных книг model_advanced_test: {mean_ap_popular_model_advabced_test2:.4f}")

Результаты эксперементов для случайного бейзлайна без учета рейтингов книг
**********
Средний AP@10 для случайного бейзлайна: 0.0000
Средний AP@10 для случайного бейзлайна matrix: 0.0023
Средний AP@10 для случайного бейзлайна на matrix_test: 0.0025
Средний AP@10 для случайного бейзлайна model_base_train: 0.0135
Средний AP@10 для случайного бейзлайна model_base_test: 0.0023
Средний AP@10 для случайного бейзлайна model_advanced_train: 0.0216
Средний AP@10 для случайного бейзлайна model_advanced_test: 0.0276
**********
Результаты эксперементов для бейзлайна на основе популярных книг
**********
Средний AP@10 для бейзлайна на основе популярных книг: 0.1595
Средний AP@10 для бейзлайна на основе популярных книг matrix: 0.1511
Средний AP@10 для бейзлайна на основе популярных книг test: 0.1516
Средний AP@10 для бейзлайна на основе популярных книг model_base_train: 0.0135
Средний AP@10 для бейзлайна на основе популярных книг model_base_test: 0.0207
Средний AP@10 для бейзлайна на основе популярны

### Случайный бейзлайн.


Значение AP@10 для случайного бейзлайна (0.0000) указывает на то, что ни одна из рекомендованных книг не совпала с релевантными книгами, что вполне ожидаемо для случайных рекомендаций.

Значения AP@10 для разреженной матрицы (matrix) и matrix_test (по 0.0015) показывают небольшое улучшение, но все еще очень низкие результаты, что говорит о том, что даже в этих случаях рекомендации не соответствуют ожиданиям пользователей.
    
Показатели model_base_train и model_advanced_train значительно выше, что указывает на то, что обученные модели могут предоставлять более релевантные рекомендации по сравнению с простыми случайными рекомендациями.


### Бейзлайн на основе популярных книг.


Значение AP@10 для бейзлайна на основе популярных книг (0.1449) значительно выше, чем у случайного бейзлайна, что подтверждает эффективность использования популярности книг как стратегии рекомендаций.
    
Результаты для различных разреженных матриц (matrix и test) также показывают небольшое снижение, но остаются достаточно высокими по сравнению с результатами случайного бейзлайна.
    
Показатели для обученных моделей (model_base и model_advanced) остаются низкими по сравнению с популярными рекомендациями, но их значения все же выше, чем у случайного подхода.


Использование популярности книг как метода рекомендации показывает значительное улучшение по сравнению с случайным методом, что подчеркивает важность учета исторических данных о предпочтениях пользователей.